In [ ]:
FileName: /home/tivo/Desktop/songs/wav/JDSC.wav 
nsamps: 11556864
fs: 44100
shape tmdata: (11556864, 2) tmprate: 44100
shape samps: 11556864

In [341]:
import wave, array, math, time, argparse, sys
import numpy as np, pywt
from scipy import signal
import pdb
import matplotlib.pyplot as plt
import pandas as pd
from scipy.io import wavfile 

def read_wav(filename):
    return wavfile.read(filename)

'''
    #open file, get metadata for audio
    try:
        wf = wave.open(filename,'rb')
    except IOError as e:
        print (e)
        return

    # typ = choose_type( wf.getsampwidth() ) #TODO: implement choose_type
    nsamps = wf.getnframes();
    assert(nsamps > 0);
    print ("nsamps: {}".format(nsamps))
    
    fs = wf.getframerate()
    assert(fs > 0)
    print ("fs: {}".format(fs))
    
    tmprate, tmpdata = wavfile.read(filename)
    print("shape tmdata: {} tmprate: {}".format(tmpdata.shape,tmprate))
''' 

    
'''    
    # read entire file and make into an array
    samps = list(array.array('i',wf.readframes(nsamps)))
    #print 'Read', nsamps,'samples from', filename
    try:
        assert(nsamps == len(samps))
    except AssertionError as e:
        print  (nsamps, "not equal to", len(samps))
    print ("shape samps: {}".format(len(samps)))
    return samps, fs
''' 

def read_wav_mono(filename):

    #open file, get metadata for audio
    try:
        wf = wave.open(filename,'rb')
    except IOError as e:
        print (e)
        return

    # typ = choose_type( wf.getsampwidth() ) #TODO: implement choose_type
    nsamps = wf.getnframes();
    assert(nsamps > 0);
    #print ("nsamps: {}".format(nsamps))
    
    fs = wf.getframerate()
    assert(fs > 0)
    #print ("fs: {}".format(fs))
    
    tmprate, tmpdata = wavfile.read(filename)
    #print("shape tmdata: {} tmprate: {}".format(tmpdata.shape,tmprate))
       
    # read entire file and make into an array
    samps = list(array.array('i',wf.readframes(nsamps)))
    #print 'Read', nsamps,'samples from', filename
    try:
        assert(nsamps == len(samps))
    except AssertionError as e:
        print  (nsamps, "not equal to", len(samps))
    #print ("shape samps: {}".format(len(samps)))
    return samps, fs


    #sampsarray = np.zeroes(tmpdata.shape)
    #for i in tmpdata.shape[1]:
    #amps = list(array.array('i',wf.readframes(nsamps)))
    #sampsarray = np.column_stack(tmpdata[0],tmpdata[1])
    
    

def ab_adjuster(value, granualityRoundOff):
    #granualityRoundOff means : if granualityRoundOff is 5, then 12.4 would be rounded to 10, 
    # 12.6 would be rounded to 15
    quotient,remainder = divmod(value,granualityRoundOff)
    #remainder = value%granualityRoundOff
    #print (quotient)
    if ( float(remainder) < float(granualityRoundOff)/2):
        #print(quotient*granualityRoundOff)
        return quotient*granualityRoundOff
    else:
        #print((quotient+1)*granualityRoundOff)
        return ((quotient+1)*granualityRoundOff)


# print an error when no data can be found
def no_audio_data():
    print ("No audio data for sample, skipping...")
    return None, None
    
# simple peak detection
def peak_detect(data):
    max_val = numpy.amax(abs(data)) 
    peak_ndx = numpy.where(data==max_val)
    if len(peak_ndx[0]) == 0: #if nothing found then the max must be negative
        peak_ndx = numpy.where(data==-max_val)
    return peak_ndx

def my_print(toprint):
    printEnable = True
    if (printEnable):
        print(toprint)
def bpm_detector(data,fs):
    #print("")
    #print ("+"*80)
    cA = [] 
    cD = []
    correl = []
    cD_sum = []
    levels = 4
    max_decimation = 2**(levels-1);
    min_ndx = 60./ 220 * (fs/max_decimation)
    max_ndx = 60./ 40 * (fs/max_decimation)
    
    print("type(Data): {} shape(Data): {}".format(type(data),len(data)))
    #data = data.tolist()
    for loop in range(0,levels):
        cD = []
        # 1) DWT
        if loop == 0:
            [cA,cD] = pywt.dwt(data,'db4');
            #cA : approximation coeff
            #cD : Detailed Coeff
            cD_minlen = len(cD)/max_decimation+1;
            cD_sum = numpy.zeros(cD_minlen);
        else:
            [cA,cD] = pywt.dwt(cA,'db4');
        
        #print("cA : {}, cD: {}".format(cA,cD))
        
        #print ("DWT : type(cA) : {} type(cD): {} len(cA) {}, len(cD) {}".format(type(cA), type(cD), cA.shape, cD.shape))
        # 2) Filter
        cD = signal.lfilter([0.01],[1 -0.99],cD);
        #print("Filtered cD")
        #print(cD)

        # 4) Subtractargs.filename out the mean.

        # 5) Decimate for reconstruction later.
        cD = abs(cD[::(2**(levels-loop-1))]);
        #print("Cd raise to level-inverse{}",format(cD))
        #print("cD shape: {}".format(cD.shape))
        cD = cD - numpy.mean(cD);
        #print("cD - mean(cD)")
        #print(cD)
        # 6) Recombine the signal before ACF
        #    essentially, each level I concatenate 
        #    the detail coefs (i.e. the HPF values)
        #    to the beginning of the array
        cD_sum = cD[0:cD_minlen] + cD_sum;
        #print ("cD_sum")
        #print(cD_sum)
    
    #print ("Loop over")
    if [b for b in cA if b != 0.0] == []:
        return no_audio_data()
    # adding in the approximate data as well...    
    cA = signal.lfilter([0.01],[1 -0.99],cA);
    cA = abs(cA);
    cA = cA - numpy.mean(cA);
    #print("cA - mean(cA){}".format(cA))
    
    #print ("Filtered cA{}".format(cA))
    cD_sum = cA[0:cD_minlen] + cD_sum;
    #print ("cD_sum_with-approx-cA")
    #print(cD_sum)
    
    # ACF
    correl = numpy.correlate(cD_sum,cD_sum,'full') 
    #print("sizeof(cD_sum_with-approx-cA){}, sizeof(ACF){}".format(len(cD_sum),len(correl)))
    #print ('ACF on cD_sum_with-approx-cA')
    #print (correl)
    
    midpoint = len(correl) / 2
    correl_midpoint_tmp = correl[midpoint:]
    #print ('correl_midpoint_tmp')
    #print (correl_midpoint_tmp)
    #print ("min_ndx{} max_ndx{}".format(min_ndx,max_ndx))
    peak_ndx = peak_detect(correl_midpoint_tmp[min_ndx:max_ndx]);
    if len(peak_ndx) > 1:
        return no_audio_data()
    
    #print ('peak_ndx')
    #print (peak_ndx)
    peak_ndx_adjusted = peak_ndx[0]+min_ndx;
    #print ('peak_ndx_adjusted,max_decimation')
    #print (peak_ndx_adjusted,max_decimation)
    bpm = 60./ peak_ndx_adjusted * (fs/max_decimation)
    print (bpm)
    return bpm,correl
    
    


In [342]:
import pywt
#bpm_detector(range(1,10),5)
#(cA, cD) = pywt.dwt([1, 2, 3, 4, 5, 6], 'db1')

In [343]:
bpmsDF = pd.DataFrame(columns=['audioFile','bpms'])

def extractBPMtoDF(filename,audioWindowinSec):
    #filename = "/home/tivo/Desktop/songs/wav/AYTH.wav"
    #window = 5
    window = audioWindowinSec
    global bpmsDF
    
    fs,samps = read_wav(filename)
    
    data = []
    correl=[]
    bpm = 0
    n=0;
    nsamps = samps.shape[0]
    print("samp shape: {}, row: {}, col: {}".format(samps.shape,samps.shape[0],samps.shape[1]))
    window_samps = int(window*fs)         
    samps_ndx = 0;  #first sample in window_ndx 
    max_window_ndx = nsamps / window_samps;
    bpms = numpy.zeros(max_window_ndx)
    adjusted_bpms = []

    print ("fs {}, nsamps {}, window_samps {},max_window_ndx {}, int(max_window_ndx) {}"\
           .format(fs, nsamps, window_samps,max_window_ndx, int(max_window_ndx)))
    #iterate through all windows
    for channelIdx in range(0,samps.shape[1]):
        print()
        sampList = list(samps[:,channelIdx])
        for window_ndx in range(0,int(max_window_ndx)):

            #get a new set of samples
            #print n,":",len(bpms),":",max_window_ndx,":",fs,":",nsamps,":",samps_ndx
            print("channelIdx: {}",format(channelIdx))
            print(len(sampList))
            #data = list(samps[channelIdx:,].tolist()[samps_ndx:samps_ndx+window_samps])
            data = sampList[samps_ndx:samps_ndx+window_samps]
            print("type(Data): {}".format(type(data)))
            print("Data-size: {}".format(len(data)))
            if not ((len(data) % window_samps) == 0):
                print("Skipping channelIdx :{}".format(channelIdx))
                continue
                #raise AssertionError( str(len(data) ) ) 

            bpm, correl_temp = bpm_detector(data,fs)

            if bpm == None:
                continue

            adjusted_bpm = ab_adjuster(bpm,5) # 5 second is round off granuality
            bpms[window_ndx] = bpm
            adjusted_bpms.append(str(int(adjusted_bpm)))
            correl = correl_temp

            #iterate at the end of the loop
            samps_ndx = samps_ndx+window_samps;
            n=n+1; #counter for debug...

        bpmsDF = bpmsDF.append(pd.DataFrame({'audioFile': filename,'bpms': ' '.join(adjusted_bpms)}, index=[0]))
        bpm = numpy.median(bpms)
        #print ('Completed.  Estimated Beats Per Minute:', bpm)

        n = range(0,len(correl))
        plt.plot(n,abs(correl)); 
        plt.show(False); #plot non-blocking
        time.sleep(10);
        plt.close();


def extractBPMtoDFExisting(filename,audioWindowinSec):
    #filename = "/home/tivo/Desktop/songs/wav/AYTH.wav"
    #window = 5
    window = audioWindowinSec
    global bpmsDF
    
    samps,fs = read_wav_mono(filename)
    
    data = []
    correl=[]
    bpm = 0
    n=0;
    nsamps = len(samps)
    window_samps = int(window*fs)         
    samps_ndx = 0;  #first sample in window_ndx 
    max_window_ndx = nsamps / window_samps;
    bpms = numpy.zeros(max_window_ndx)
    adjusted_bpms = []

    print ("fs {}, nsamps {}, window_samps {},max_window_ndx {}, int(max_window_ndx) {}"\
           .format(fs, nsamps, window_samps,max_window_ndx, int(max_window_ndx)))
    #iterate through all windows
    for window_ndx in range(0,int(max_window_ndx)):

        #get a new set of samples
        #print n,":",len(bpms),":",max_window_ndx,":",fs,":",nsamps,":",samps_ndx
        data = samps[samps_ndx:samps_ndx+window_samps]
        print("type(Data): {}".format(type(data)))
        print("Data-size: {}".format(len(data)))
        if not ((len(data) % window_samps) == 0):
            raise AssertionError( str(len(data) ) ) 
        
        bpm, correl_temp = bpm_detector(data,fs)

        if bpm == None:
            continue
        
        adjusted_bpm = ab_adjuster(bpm,5) # 5 second is round off granuality
        bpms[window_ndx] = bpm
        adjusted_bpms.append(str(int(adjusted_bpm)))
        correl = correl_temp
        
        #iterate at the end of the loop
        samps_ndx = samps_ndx+window_samps;
        n=n+1; #counter for debug...

    bpmsDF = bpmsDF.append(pd.DataFrame({'audioFile': filename,'bpms': ' '.join(adjusted_bpms)}, index=[0]))
    bpm = numpy.median(bpms)
    #print ('Completed.  Estimated Beats Per Minute:', bpm)
    
    n = range(0,len(correl))
    plt.plot(n,abs(correl)); 
    plt.show(False); #plot non-blocking
    time.sleep(10);
    plt.close();


In [344]:
bpmsDF

,audioFile,bpms


In [345]:
import os
from os.path import basename
import argparse

'''
parser = argparse.ArgumentParser(description='Process mp3dir and convert all files in dir to wav')
parser.add_argument('--wavdir', required=True,
                   help='wavDir directory absolute path where all wavs for BPM are kept')
parser.add_argument('--csvFilePath', required=True,
                   help='wavFileName(column1),bpms(column2) space seprated would be saved here')


args = parser.parse_args()
wavDir = args.wavdir
csvFilePath = args.csvFilePath
'''
wavDir = '/home/tivo/Desktop/songs/wav'
csvFilePath = '/home/tivo/Desktop/bpms.csv'


for filename in os.listdir(wavDir):
    #print (wavDir, filename)
    wavFilePath = "".join([wavDir, "/",filename])
    print("FileName: {} ".format(wavFilePath))
    extractBPMtoDFExisting(wavFilePath,120)
    print("=="*80)
    extractBPMtoDF(wavFilePath,120)

bpmsDF.to_csv(csvFilePath)


FileName: /home/tivo/Desktop/songs/wav/QOSS.wav 
fs 44100, nsamps 10544256, window_samps 5292000,max_window_ndx 1.9924897959183674, int(max_window_ndx) 1


/home/tivo/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:84: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


type(Data): <class 'list'>
Data-size: 5292000
type(Data): <class 'list'> shape(Data): 5292000


/home/tivo/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:138: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/tivo/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:163: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/tivo/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:177: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/tivo/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:188: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/tivo/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:192: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/tivo/anaconda3/lib/python3.5/site-packages/i

[ 42.49969336]
samp shape: (10544256, 2), row: 10544256, col: 2
fs 44100, nsamps 10544256, window_samps 5292000,max_window_ndx 1.9924897959183674, int(max_window_ndx) 1



/home/tivo/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:20: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


channelIdx: {} 0
10544256
type(Data): <class 'list'>
Data-size: 5292000
type(Data): <class 'list'> shape(Data): 5292000
[ 42.49969336]


/home/tivo/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:45: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.



channelIdx: {} 1
10544256
type(Data): <class 'list'>
Data-size: 5252256
Skipping channelIdx :1
FileName: /home/tivo/Desktop/songs/wav/JDSC.wav 
fs 44100, nsamps 11556864, window_samps 5292000,max_window_ndx 2.1838367346938776, int(max_window_ndx) 2
type(Data): <class 'list'>
Data-size: 5292000
type(Data): <class 'list'> shape(Data): 5292000
[ 42.99693322]
type(Data): <class 'list'>
Data-size: 5292000
type(Data): <class 'list'> shape(Data): 5292000
[ 42.99693322]
samp shape: (11556864, 2), row: 11556864, col: 2
fs 44100, nsamps 11556864, window_samps 5292000,max_window_ndx 2.1838367346938776, int(max_window_ndx) 2

channelIdx: {} 0
11556864
type(Data): <class 'list'>
Data-size: 5292000
type(Data): <class 'list'> shape(Data): 5292000
[ 42.99693322]
channelIdx: {} 0
11556864
type(Data): <class 'list'>
Data-size: 5292000
type(Data): <class 'list'> shape(Data): 5292000
[ 42.99693322]

channelIdx: {} 1
11556864
type(Data): <class 'list'>
Data-size: 972864
Skipping channelIdx :1
channelIdx: 

KeyboardInterrupt: 

In [ ]:
test = numpy.array([[1, 2],[3,4], [3, 4],[5,6]])

In [303]:
print(test.shape)
print(test[1:].shape)
print(test[1:,1])

(4, 2)
(3, 2)
[4 4 6]


In [304]:
test[:,0].tolist()

[1, 3, 3, 5]